In [1]:
from __future__ import division
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.layers import dropout

In [2]:
#load data
UserTargdayLabel = pd.read_csv('C:/Users/Administrator.NBJXUEJUN-LI/Desktop/project/RProj/my code/r相关函数/normaldnn/save/UserTargdayLabel.csv',
                      header=0,encoding='gbk' )

In [3]:
#perpare datas
predData  = UserTargdayLabel[UserTargdayLabel['label']==-1]
trainData = UserTargdayLabel[UserTargdayLabel['label']!=-1]
predX     = np.array(predData.ix[:,3:],dtype=np.float32)
allX      = np.array(trainData.ix[:,3:],dtype=np.float32)
ally      = np.array(trainData.ix[:,2],dtype=np.float32)
ally = ally.reshape((len(ally),1))
trainSampNums = int(len(allX)*0.8)
colNums       =len(allX[0])
trainX     = allX[:trainSampNums]
trainy     = ally[:trainSampNums]
testX      = allX[trainSampNums:]
testy      = ally[trainSampNums:]


In [4]:
#set neurons' dimemsions
n_inputs  = colNums
n_hidden1 = n_inputs//2
n_hidden2 = n_hidden1//4
n_hidden3 = n_hidden2//8
n_outpus  = 1


In [5]:
keep_prob = 0.7
with tf.name_scope("dnn"):  
    is_training = tf.placeholder_with_default(False, shape=(), name='is_training') 
    X = tf.placeholder(tf.float32, shape=[None, n_inputs])
    y = tf.placeholder(tf.float32, shape=[None, n_outpus])
    X_drop = dropout(X, keep_prob, is_training=is_training)
    hidden1 = fully_connected(X_drop, n_hidden1,  scope="hidden1",activation_fn=tf.nn.tanh)
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2",activation_fn=tf.nn.tanh)
    hidden3 = fully_connected(hidden2, n_hidden3, scope="hidden3",activation_fn=tf.nn.tanh)
    logits  = fully_connected(hidden2, n_outpus, scope="outputs", activation_fn=None)

In [6]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [7]:
learning_rate = 0.0001
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [8]:
with tf.name_scope("eval"):
    #correct = tf.nn.in_top_k(logits, y, 1)
    #accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    prediction = tf.round(tf.sigmoid(logits))
    predictions_correct = tf.cast(tf.equal(prediction, y),tf.float32)
    accuracy = tf.reduce_mean(predictions_correct)

In [9]:
init = tf.global_variables_initializer()
n_epochs = 3000
batch_size = 1000
batch_num = len(trainX) // batch_size+1
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):  
        for batch in range(batch_num):
            X_batch, y_batch = trainX[batch*batch_size:(batch+1)*batch_size],trainy[batch*batch_size:(batch+1)*batch_size]
            sess.run(training_op, feed_dict={X: X_batch,y:y_batch,is_training:True})
        if epoch%100==0:
            train_accu = accuracy.eval(feed_dict={X: trainX,y:trainy})
            test_accu  = accuracy.eval(feed_dict={X: testX, y:testy})
            print(train_accu,test_accu)


0.271825 0.206349
0.694444 0.738095
0.777778 0.793651
0.827381 0.81746
0.833333 0.849206
0.84127 0.873016
0.843254 0.880952
0.843254 0.880952
0.847222 0.880952
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
0.847222 0.888889
